In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [90]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
def get_clf_eval(y_test, y_pred):
    confusion = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred)
    AUC = roc_auc_score(y_test, y_pred)
    print('오차행렬:\n', confusion)
    print('\n정확도: {:.4f}'.format(accuracy))
    print('정밀도: {:.4f}'.format(precision))
    print('재현율: {:.4f}'.format(recall))
    print('F1: {:.4f}'.format(F1))
    print('AUC: {:.4f}'.format(AUC))

In [55]:
health = pd.read_csv('최종데이터.csv',encoding='cp949')
#FBS 110 이하 = 정상 / 110 이상 = 양성
for i in range(len(health)):
    if health['FBS'][i]<126:
        health['FBS'][i]=0
    else: 
        health['FBS'][i]=1

<ipython-input-55-8b8873bdb7af>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  health['FBS'][i]=0
<ipython-input-55-8b8873bdb7af>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  health['FBS'][i]=1


In [56]:
health_target = health[['FBS']]
health_data = health[health.columns.difference(['FBS'])]

health_data = np.array(health_data)
health_target = np.array(health_target)

In [84]:
# test data split
X_trainval, X_test, y_trainval, y_test = train_test_split(health_data, health_target,random_state=42)

In [87]:
#standardScaler 사용
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_trainval)
X_trainval_scaled = scaler.transform(X_trainval) 
X_test_scaled = scaler.transform(X_test)

In [88]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression

In [124]:
#스케일링 했을때 + balanced
scaler = StandardScaler()

best_score = 0
for C in [0.001, 0.01, 0.1, 1, 10, 100]:
    for penalty in ['l2']:
        
        score_val = []
        for train_idx , val_idx in kfold.split(X_trainval,y_trainval):
            
            X_train = X_trainval[train_idx]
            y_train = y_trainval[train_idx]
            X_valid = X_trainval[val_idx]
            y_valid = y_trainval[val_idx]
            
            
            scaler.fit(X_train)
            X_train_scaled = scaler.transform(X_train)
            X_valid_scaled = scaler.transform(X_valid)
            logreg = LogisticRegression(C=C,penalty=penalty,class_weight='balanced')
            
            logreg.fit(X_train_scaled, y_train)
            
            y_valid_hat = logreg.predict(X_valid_scaled)
            
            score_val.append(recall_score(y_valid,y_valid_hat))
            
        mean_score = np.mean(score_val)
        
        if mean_score > best_score:
            best_score = mean_score
            best_parameters = {'C':C,'penalty':penalty,'class_weight':'balanced'}
            
            
print("Best score on validation set: {:.4f}".format(best_score))
print("Best hyperparameters: {}".format(best_parameters))



C:\Users\choib\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\choib\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\choib\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\choib\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using r

Best score on validation set: 0.70
Best hyperparameters: {'C': 0.001, 'penalty': 'l2', 'class_weight': 'balanced'}


In [125]:
best_logreg = LogisticRegression(**best_parameters)

X_trainval_scaled = scaler.transform(X_trainval)
X_test_scaled = scaler.transform(X_test)
best_logreg.fit(X_trainval_scaled,y_trainval)


y_test_hat = best_logreg.predict(X_test_scaled)


get_clf_eval(y_test,y_test_hat)

C:\Users\choib\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


오차행렬:
 [[60242 27765]
 [ 3132  6902]]

정확도: 0.6849
정밀도: 0.1991
재현율: 0.6879
F1: 0.3088
AUC: 0.6862


In [118]:
#scaling 안했을때
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)
params_grid = {'C':[0.001, 0.01, 0.1, 1, 10, 100],'class_weight':['balanced']}

grid_search = GridSearchCV(LogisticRegression(),param_grid=params_grid,scoring='recall',refit=True,cv=kfold)
grid_search.fit(X_trainval,y_trainval)

print("Best score on validation set: {:.4f}".format(grid_search.best_score_))
print("Best hyperparameters: {}".format(grid_search.best_params_))


C:\Users\choib\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\choib\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\choib\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\choib\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using r

C:\Users\choib\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\choib\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\choib\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\choib\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using r

Best score on validation set: 0.63
Best hyperparameters: {'C': 0.01, 'class_weight': 'balanced'}


In [119]:
best_logreg=grid_search.best_estimator_

In [121]:
best_logreg.fit(X_trainval,y_trainval)
y_pred = best_logreg.predict(X_test)

get_clf_eval(y_test,y_pred)

C:\Users\choib\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


오차행렬:
 [[58099 29908]
 [ 3274  6760]]

정확도: 0.6615
정밀도: 0.1844
재현율: 0.6737
F1: 0.2895
AUC: 0.6669


In [122]:
#scaling 안했을때 + balanced weight 적용 x
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)
params_grid = {'C':[0.001, 0.01, 0.1, 1, 10, 100]}

grid_search = GridSearchCV(LogisticRegression(),param_grid=params_grid,scoring='recall',refit=True,cv=kfold)
grid_search.fit(X_trainval,y_trainval)

print("Best score on validation set: {:.2f}".format(grid_search.best_score_))
print("Best hyperparameters: {}".format(grid_search.best_params_))


C:\Users\choib\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\choib\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\choib\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\choib\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using r

Best score on validation set: 0.01
Best hyperparameters: {'C': 0.001}


In [123]:
best_logreg=grid_search.best_estimator_
best_logreg.fit(X_trainval,y_trainval)
y_pred = best_logreg.predict(X_test)

get_clf_eval(y_test,y_pred)

C:\Users\choib\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


오차행렬:
 [[87828   179]
 [ 9932   102]]

정확도: 0.8969
정밀도: 0.3630
재현율: 0.0102
F1: 0.0198
AUC: 0.5041


정확도는 올라가지만, recall이 너무 낮아짐 -> f1score 매우 낮아짐...

# 모델 선정

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(health_data, health_target, stratify=health_target,random_state=42)

In [ ]:
from sklearn.model_selection import GridSearchCV
from scipy.stats import uniform, randint
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
models = [
    ('LogisticRegression', LogisticRegression()),
    ('RandomForestClassifier', RandomForestClassifier()),
    ('SVM',SVC())
]
n = 3
params = {
    'LogisticRegression': {
        'C': [0.001, 0.01, 0.1, 1.0, 10, 100],
        'class_weight':['balanced',None],
    },
    'RandomForestClassifier':{
        'n_estimators': [1,2,3,5,10,20,50,100,300],
        'max_depth': [1,3,5,10,20],
        'min_samples_leaf': [1,2,5,10,20],
        'class_weight':['balanced',None]
    },
    'SVM':{
        'gamma':[2**-15,2**-13,2**-11,2**-9,2**-7,2**-5,2**-3,2**-1,2**1,2**3],
        'C':[2**-5,2**-3,2**-1,2**1,2**3,2**5,2**7,2**9,2**11,2**13,2**15]
    }
    
}
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)
best_model, best_recall_score = None, 0.0
for model_name, model in models:
    param_grid = params[model_name]
    grid = GridSearchCV(model, cv=kfold, n_jobs=-1 ,param_grid=param_grid)
    grid = grid.fit(X_trainval, y_trainval)

    model = grid.best_estimator_
    predictions = model.predict(X_test)
    recall = recall_score(y_test, predictions)

    print(model_name, recall)

    if recall < best_recall_score:
        best_model = model

In [ ]:
'RandomForestClassifier':{
        'n_estimators': [1,2,3,5,10,20,50,100,300],
        'max_depth': [1,3,5,10,20],
        'min_samples_leaf': [1,2,5,10,20],
        'class_weight':['balanced',None]
    },

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

X_trainval, X_test, y_trainval, y_test = train_test_split(
    health_data, health_target, test_size=0.25, random_state=0)

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=2)

In [ ]:
hyperparam_grid = {'gamma':[2**-15,2**-13,2**-11,2**-9,2**-7,2**-5,2**-3,2**-1,2**1,2**3],
        'C':[2**-5,2**-3,2**-1,2**1,2**3,2**5,2**7,2**9,2**11,2**13,2**15]}
grid_search = GridSearchCV(SVC(), hyperparam_grid, scoring='accuracy', refit=True, cv=kfold)
grid_search.fit(X_trainval, y_trainval)

print("Best score on validation set: {:.2f}".format(grid_search.best_score_))
print("Best hyperparameters: {}".format(grid_search.best_params_))